# SMU Logo bounding box detection
Design an algorithm to identify all images of SMU from a set of random images. A bounding box should be drawn around the SMU logo whenever it appears in an image.

When an SMU logo occurs in an image, a bounding box should be drawn around it. Logo identification will be evaluated by its F1 score.

### Setup
Install dependencies
```bash
pip install -r requirements.txt

```

In [1]:
%pip install -r requirements.txt

^C
Note: you may need to restart the kernel to use updated packages.


Import packages

In [1]:
from ultralytics import YOLO
from roboflow import Roboflow
from PIL import Image
from dotenv import load_dotenv
from manage_model import update_path, convert_bbox_format, get_f1
import os

In [2]:
# Project settings
load_dotenv()
api_key = os.getenv('API_KEY')
data_version = os.getenv('DATA_VERSION')
path_to_project = os.getenv("PATH_TO_PROJECT")

# If on SMU Violet, can't load env variables. Write manually.

path_to_data = f'{path_to_project}/tree-type-detection-{data_version}'
print(path_to_data)

D:/Disk D Documents/CS424/Project SMU Logo/tree-type-detection-17


Load dataset into workspace

In [3]:
rf = Roboflow(api_key=api_key)
project = rf.workspace("tree-dataset-iftyz").project("tree-type-detection-9rfxy")
dataset = project.version(data_version).download("yolov8")
print(f'Data version: {data_version}')

loading Roboflow workspace...
loading Roboflow project...
Data version: 17


Update test, train and val values in data.yaml
This fixes a file not found bug

In [4]:
update_path(data_version, path_to_project)

In [5]:
# Use saved model
model = YOLO('./saved_models/v17_epoch64.pt')

After training, the model can be found in `/runs`. Save the `best.pt` model in the `/saved_models` folder.

### [Test](https://docs.ultralytics.com/modes/predict/#why-use-ultralytics-yolo-for-inference)
Predict bounding boxes on test set

In [10]:
# Predict
# prediction = model.predict(source=f"{path_to_data}/test/images", conf=0.25, iou=0.6)
prediction = model.predict(source=f"{path_to_data}/../image_folder", conf=0.4, iou=0.6)


image 1/14 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-17\..\image_folder\DSCF1293.JPG: 448x640 (no detections), 18.5ms
image 2/14 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-17\..\image_folder\DSCF1319.JPG: 448x640 (no detections), 14.5ms
image 3/14 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-17\..\image_folder\DSCF1677.JPG: 640x448 (no detections), 16.5ms
image 4/14 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-17\..\image_folder\DSCF1708.JPG: 640x448 (no detections), 117.5ms
image 5/14 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-17\..\image_folder\DSCF1722.JPG: 448x640 (no detections), 286.5ms
image 6/14 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-17\..\image_folder\DSCF1761.JPG: 384x640 2 SMU-Logos, 15.5ms
image 7/14 D:\Disk D Documents\CS424\Project SMU Logo\tree-type-detection-17\..\image_folder\DSCF1897.JPG: 640x544 3 SMU-Logos, 17.5ms
image 8/14 D:\Disk D Documents\C

Save images to `/predictions` folder if bounding box exists.

In [11]:
images = {}

if not os.path.exists(f'{path_to_data}/../predictions'):
    os.makedirs(f'{path_to_data}/../predictions')

for i in range(0, len(prediction)):
    if prediction[i].boxes.shape[0] >= 1:
        im_rgb = Image.fromarray(prediction[i].plot(line_width=1)[..., ::-1])
        images[i] = im_rgb  # RGB-order PIL image
        boxes = prediction[i].boxes.xyxy.tolist()
        image_name = prediction[i].path.split("\\")[-1]
        predicted_boxes = []
        for i, b in enumerate(boxes):
            predicted_box = convert_bbox_format(b, image_name)
            predicted_boxes.append(predicted_box)

            # Write labels
            with open(f'{path_to_data}/../predictions/object_est.txt', 'a') as f:
                f.write(', '.join(map(str, predicted_box)) + '\n')
        # Save images
        im_rgb.save(fp=f'{path_to_data}/../predictions/{image_name}_labelled.jpg')

len(images)

7